# Costs and Time Overviews
## Costs
* Started with $5.58  
* Ended up with $5.32 after 100 books 
* So $0,26 per 100 books
* So for 17,000 books = $44,2

## Time
* 100 books take around 5 minutes, 
* so 17,000 books will take around 14 hours

Generell festzuhalten: Sumscore und proportions zeigen im grunde das gleiche Muster. 

Sum Score: 
------------------------

                pos_sum   neg_sum  total_sum
author_gender                               
0              3.233743  5.510914  -2.277171
1              4.102130  5.795434  -1.693303


------------------------
POS SUM
author_gender              0         1
master_genre                          
Children            2.537037  3.338645
Fantasy             3.505495  4.297003
Historical Fiction  4.096916  4.862588
Horror              3.013043  3.560976
Literary Fiction    3.372059  3.793333
Mystery             2.712018  3.031215
Other               3.002488  4.102850
Romance             3.611872  4.448276
Science Fiction     2.814010  4.301538
Thriller            2.955446  3.120448
YA                  3.158824  3.723235
author_gender
0    3.161746
1    3.870919
dtype: float64
-----------------------------
NEG SUM 
author_gender              0         1
master_genre                          
Children            4.925926  4.601594
Fantasy             7.008242  7.194142
Historical Fiction  4.506608  5.253761
Horror              5.034783  5.666667
Literary Fiction    4.455882  4.837333
Mystery             5.455782  6.915273
Other               4.942786  5.423792
Romance             4.086758  5.383029
Science Fiction     6.500000  7.175385
Thriller            6.328383  6.282913
YA                  4.829412  5.127563
author_gender
0    5.279506
1    5.805586
dtype: float64
__________________________________
author_gender              0         1
master_genre                          
Children           -2.388889 -1.262948
Fantasy            -3.502747 -2.897139
Historical Fiction -0.409692 -0.391174
Horror             -2.021739 -2.105691
Literary Fiction   -1.083824 -1.044000
Mystery            -2.743764 -3.884058 ####
Other              -1.940299 -1.320942
Romance            -0.474886 -0.934753 ####
Science Fiction    -3.685990 -2.873846
Thriller           -3.372937 -3.162465
YA                 -1.670588 -1.404328
author_gender
0   -2.117760
1   -1.934668
dtype: float64

-> test for significance? 
wird bei männlichen und weiblichen authors unterschiedlich berichtet? unterschiedlich viele NAs? 

prop_pos for men would be lower if less NAs for stereotypical questions for men -> less NAs means less stereotypical stuff that there is no evidence for 
prop_pos for women would be higher if more NAs for stereotypical questions for women -> more NAs means more stereotypical stuff that there is no evidence for

But when I looked at sum scores, where NAs are just not counted, female author also had a higher sum score: 
pos_sum   neg_sum  total_sum
author_gender                               
0              3.233743  5.510914  -2.277171
1              4.102130  5.795434  -1.693303

Maybe this is all explained by the fact that there are twice as many female authors as male authors and the biggest genre is romance and there women write stereotypically? 

In [ ]:
# SUM SCORE -------------------#########################

# 1) Mean prop_pos per (genre, gender)
by_genre = df.groupby(['master_genre','author_gender'])['neg_sum'].mean().unstack()
print(by_genre)

# 2) Average those genre‑specific means **equally**  
balanced = by_genre.mean(axis=0)
print(balanced)

In [ ]:
# Sum Scores
# Create sum scores 

# --- TOTAL SUM ---
df['total_sum'] = df[answer_cols].sum(axis=1, skipna=True)

# --- POSITIVE SUM (only 1s from even-numbered answer columns) ---
df['pos_sum'] = df[stereotypical_cols].apply(lambda row: (row == 1).sum(), axis=1)

# --- NEGATIVE SUM (only -1s from odd-numbered answer columns) ---
df['neg_sum'] = df[anti_stereotypical_cols].apply(lambda row: (row == -1).sum(), axis=1)

In [ ]:
# Create averages

# --- TOTAL AVERAGE ---
df['total_avg'] = df[answer_cols].mean(axis=1, skipna=True)

# --- POSITIVE SUM (only 1s from even-numbered answer columns) ---
df['pos_avg'] = df[stereotypical_cols].apply(lambda row: (row == 1).mean(), axis=1)

# --- NEGATIVE SUM (only -1s from odd-numbered answer columns) ---
df['neg_avg'] = df[anti_stereotypical_cols].apply(lambda row: (row == -1).mean(), axis=1)

In [ ]:
# # Find 20 novels that I've read to check manually if FMC extraction works

# # Original data with an extra column
# books = [
#     {"Title": "Pride and Prejudice", "Author": "Jane Austen", "Female_Major_Character": 1},  # Elizabeth Bennet
#     {"Title": "To Kill a Mockingbird", "Author": "Harper Lee", "Female_Major_Character": 1},  # Scout Finch
#     {"Title": "The Book Thief", "Author": "Markus Zusak", "Female_Major_Character": 1},  # Liesel Meminger
#     {"Title": "Twilight", "Author": "Stephenie Meyer", "Female_Major_Character": 1},  # Bella Swan
#     {"Title": "The Fault in Our Stars", "Author": "John Green", "Female_Major_Character": 1},  # Hazel Grace Lancaster
#     {"Title": "The Perks of Being a Wallflower", "Author": "Stephen Chbosky", "Female_Major_Character": 1},  # Sam
#     {"Title": "Brave New World", "Author": "Aldous Huxley", "Female_Major_Character": 1}, #Lenina Crowne
#     {"Title": "A Game of Thrones (A Song of Ice and Fire, #1)", "Author": "George R.R. Martin", "Female_Major_Character": 1},  # Daenerys Targaryen, Arya Stark, Cersei Lannister, ...
#     {"Title": "The Lightning Thief", "Author": "Rick Riordan", "Female_Major_Character": 1},  # Annabeth Chase
#     {"Title": "Lolita", "Author": "Vladimir Nabokov", "Female_Major_Character": 1},  # Dolores "Lolita" Haze

#     {"Title": "The Hobbit and The Lord of the Rings", "Author": "J.R.R. Tolkien", "Female_Major_Character": 0},
#     {"Title": "The Little Prince", "Author": "Antoine de Saint-Exupéry", "Female_Major_Character": 0},
#     {"Title": "Crime and Punishment", "Author": "Fyodor Dostoevsky", "Female_Major_Character": 0},  
#     {"Title": "Lord of the Flies", "Author": "William Golding", "Female_Major_Character": 0}, 
#     {"Title": "The Adventures of Huckleberry Finn", "Author": "Mark Twain", "Female_Major_Character": 0}, 
#     {"Title": "The Old Man and the Sea", "Author": "Ernest Hemingway", "Female_Major_Character": 0}, 
#     {"Title": "Alex Rider", "Author": "Anthony Horowitz", "Female_Major_Character": 0},  
#     {"Title": "Der Trafikant", "Author": "Robert Seethaler", "Female_Major_Character": 0}, 
# ]

# # Create the DataFrame
# df_books = pd.DataFrame(books)

# # Display the updated DataFrame
# print(df_books)

In [ ]:
# Iterating through books, 1 book = 1 request
for index, row in df100.iterrows():
    # Extract title and author from each row 
    title = row["title"]
    author = row["author"]

    # Format user prompt with current title and author
    user_prompt = user_prompt_template.format(title=title, author=author)

    # Payload
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 200,
        "temperature": 0,  # Controls randomness in the response
        # "top_p": 0.9,
        # "search_domain_filter": ["<any>"],
        # "return_images": False,
        # "return_related_questions": False,
        # "search_recency_filter": "<string>",
        # "top_k": 0,
        # "stream": False,
        # "presence_penalty": 0,
        # "frequency_penalty": 1,
        "response_format": {
          "type": "json_schema",
          "json_schema": {"schema": AnswerFormat.model_json_schema()},
        },
        # "web_search_options": {"search_context_size": "high"}
    }

    # Send the request to the Perplexity API
    response = requests.post(url, json=payload, headers=headers)
    
    # Check the response
    if response.status_code == 200:
        response_json = response.json()
        # Extract the answer from the response
        answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')
        # Extract sources
        citation = response_json.get("citations", [])
        # Store answer and sources
        responses.append(answer)
        citations.append(citation)
    else:
        # Handle errors by appending an error message
        responses.append(f"Error: {response.status_code}")
    
    # # Sleep to avoid rate limiting
    # time.sleep(1)

# End timing the request process
end_time = time.time()

# Calculate and print the time taken for the 100 requests
elapsed_time = end_time - start_time
print(f"Time taken for 100 books: {elapsed_time} seconds")

In [ ]:
# # Specify answer format

# from pydantic import RootModel
# from typing import List, Literal, Tuple

# class AnswerFormat(RootModel):
#     root: Tuple[
#         Literal[0, 1],  # 1st: 0 or 1
#         int,            # 2nd: year
#         List[str],      # 3rd: list of genres or tags
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1]
#     ]  # Total = 29 items


In [ ]:
def validate_response_format(row):
    r = row['answer']

    # Check that it's a list and has length 29
    if not isinstance(r, list) or len(r) != 29:
        return False

    try:
        # r[0] should be 0 or 1
        if r[0] not in [0, 1]:
            return False

        # r[1] should be an int
        if not isinstance(r[1], int):
            return False

        # r[2] should be a list of strings
        if not isinstance(r[2], list):
            return False
        if not all(isinstance(tag, str) for tag in r[2]):
            return False

        # r[3:] should all be 0 or 1
        if not all(isinstance(x, int) and x in [0, 1] for x in r[3:]):
            return False

    except Exception as e:
        print(f"⚠️ Issue in row: {row['title']} — Error: {e}")
        return False

    return True

df_result['is_valid'] = df_result.apply(validate_response_format, axis=1)
invalid_rows = df_result[~df_result['is_valid']]
print(f"Number of invalid rows: {len(invalid_rows)}")
print(invalid_rows[['title', 'answer']])


In [ ]:
# # Specify answer format

# from pydantic import RootModel
# from typing import List, Literal, Tuple

# class AnswerFormat(RootModel):
#     root: Tuple[
#         Literal[0, 1, 99],  # 1st: 0 or 1
#         int,            # 2nd: year
#         List[str],      # 3rd: list of genres or tags
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1], Literal[0, 1],
#         Literal[0, 1]
#     ]  # Total = 29 items